# Preprocessing of Book-Crossing Dataset

The [Book-Crossing dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/) contains data about book ratings, books and users collected by Cai-Nicolas Ziegler in a 4-week crawl (August / September 2004).

In [1]:
import pandas as pd
import numpy as np

import functions as f

In [2]:
path = 'data/original/'

df_ratings = pd.read_csv(path + 'BX-Book-Ratings.csv', sep=';', encoding='ansi')
df_books = pd.read_csv(path + 'BX-Books.csv', sep=';', encoding='ansi', escapechar='\\')
df_users = pd.read_csv(path + 'BX-Users.csv', sep=';', encoding='ansi')

In [3]:
for df in [df_ratings, df_books, df_users]:
    df.columns = [f.colname_fix(col) for col in df.columns]

In [4]:
print('Ratings:\nNumber of ratings: %d\nNumber of books: %d\nNumber of users: %d' % (len(df_ratings),
                                                                                     len(df_ratings['isbn'].unique()),
                                                                                     len(df_ratings['user_id'].unique())))
print('\nNumber of books: %d' % len(df_books))
print('\nNumber of users: %d' % len(df_users))

Ratings:
Number of ratings: 1149780
Number of books: 340556
Number of users: 105283

Number of books: 271379

Number of users: 278858


In [5]:
f.ascii_check_bulk(df_ratings)
f.ascii_check_bulk(df_books)
f.ascii_check_bulk(df_users)

items with non-ascii characters in user_id: 0
items with non-ascii characters in isbn: 55
items with non-ascii characters in book_rating: 0

items with non-ascii characters in isbn: 0
items with non-ascii characters in book_title: 365
items with non-ascii characters in book_author: 21
items with non-ascii characters in year_of_publication: 0
items with non-ascii characters in publisher: 33
items with non-ascii characters in image_url_s: 0
items with non-ascii characters in image_url_m: 0
items with non-ascii characters in image_url_l: 0

items with non-ascii characters in user_id: 0
items with non-ascii characters in location: 560
items with non-ascii characters in age: 0



## Filtering observations
* Remove (incorrect) ISBN with non-ascii characters
* Use only country instead of whole 'location' data
* Remove images' urls
* Separate explicit (1-10) and implicit (0) ratings

In [6]:
df_ratings['isbn_check'] = df_ratings['isbn'].apply(f.ascii_check)
df_ratings = df_ratings[df_ratings['isbn_check']==0]

In [7]:
df_users['country'] = df_users['location'].apply(lambda x: x.split(', ')[-1].title())
df_users['country_check'] = df_users['country'].apply(f.ascii_check)
df_users.loc[df_users['country_check']==1, 'country'] = np.nan

In [8]:
df_ratings.drop(['isbn_check'], axis=1, inplace=True)
df_books.drop(['image_url_s', 'image_url_m', 'image_url_l'], axis=1, inplace=True)
df_users.drop(['country_check'], axis=1, inplace=True)

In [9]:
df_ratings_explicit = df_ratings[df_ratings['book_rating']!=0]
df_ratings_implicit = df_ratings[df_ratings['book_rating']==0]

print('Explicit ratings: %d\nImplicit ratings: %d' % (len(df_ratings_explicit), len(df_ratings_implicit)))

Explicit ratings: 433642
Implicit ratings: 716083


In [10]:
df_ratings_explicit.to_csv('data/ratings_explicit.csv', encoding='utf-8', index=False)
df_ratings_implicit.to_csv('data/ratings_implicit.csv', encoding='utf-8', index=False)
df_books.to_csv('data/books.csv', encoding='utf-8', index=False)
df_users.to_csv('data/users.csv', encoding='utf-8', index=False)

## Reducing the dimensionality
To reduce the dimensionality of the dataset and avoid running into memory error it will focus on users with at least 3 ratings and top 10% most frequently rated books. It consists of 176,594 records.

In [11]:
user_ratings_threshold = 3

filter_users = df_ratings_explicit['user_id'].value_counts()
filter_users_list = filter_users[filter_users >= user_ratings_threshold].index.to_list()

df_ratings_top = df_ratings_explicit[df_ratings_explicit['user_id'].isin(filter_users_list)]

print('Filter: users with at least %d ratings\nNumber of records: %d' % (user_ratings_threshold, len(df_ratings_top)))

Filter: users with at least 3 ratings
Number of records: 368563


In [12]:
book_ratings_threshold_perc = 0.1
book_ratings_threshold = len(df_ratings_top['isbn'].unique()) * book_ratings_threshold_perc

filter_books_list = df_ratings_top['isbn'].value_counts().head(int(book_ratings_threshold)).index.to_list()
df_ratings_top = df_ratings_top[df_ratings_top['isbn'].isin(filter_books_list)]

print('Filter: top %d%% most frequently rated books\nNumber of records: %d' % (book_ratings_threshold_perc*100, len(df_ratings_top)))

Filter: top 10% most frequently rated books
Number of records: 176594


In [13]:
df_ratings_top.to_csv('data/ratings_top.csv', encoding='utf-8', index=False)